# Homework 05 - Deployment

`Note: sometimes your answer doesn't match one of the options exactly. That's fine. Select the option that's closest to your solution.`

`Note: we recommend using python 3.11 in this homework.`

## Question 1

- Install Pipenv

What's the version of pipenv you installed?

`Use --version to find out`

Answer: version 2024.2.0

## Question 2

`Use Pipenv to install Scikit-Learn version 1.5.2`
    
What's the first hash for scikit-learn you get in Pipfile.lock?

`Note: you should create an empty folder for homework and do it there.`


1. pipenv --python 3.11
2. pipenv install scikit-learn==1.5.2

Answer: 03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445

### Models
We've prepared a dictionary vectorizer and a model.

### With wget:
`PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework`

`wget $PREFIX/model1.bin`

`wget $PREFIX/dv.bin`

## Question 3

Let's use these models!
- Write a script for loading these models with pickle

- Score this client:

{"job": "management", "duration": 400, "poutcome": "success"}

What's the probability that this client will get a subscription?

- 0.359
- 0.559
- 0.759
- 0.959

If you're getting errors when unpickling the files, check their checksum:

`$ md5sum model1.bin dv.bin`

`3d8bb28974e55edefa000fe38fd3ed12  model1.bin`

`7d37616e00aa80f2152b8b0511fc2dff  dv.bin`

1. pipenv shell
2. pipenv install flask
3. python3 predict.py
4. curl -X POST http://localhost:9696/predict -H "Content-Type: application/json" -d '{"job": "management", "duration": 400, "poutcome": "success"}'

if customer.json

4. curl -X POST http://localhost:9696/predict -H "Content-Type: application/json" -d @customer.json

Answer: 0.759

## Question 4

Now let's serve this model as a web service

- Install Flask and gunicorn (or waitress, if you're on Windows)
- Write Flask code for serving the model
- Now score this client using requests:

    - url = "YOUR_URL"
    - client = {"job": "student", "duration": 280, "poutcome": "failure"}
    - requests.post(url, json=client).json()

What's the probability that this client will get a subscription?

- 0.335
- 0.535
- 0.735
- 0.935

1. gunicorn --bind 0.0.0.0:9696 predict:app
2. python3 request.py

Answer: 0.334

### Docker

Install Docker. 

We will use it for the next two questions.
For these questions, we prepared a base image: `svizor/zoomcamp-model:3.11.5-slim`. You'll need to use it (see Question 5 for an example).


This image is based on python:3.11.5-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.


This is how the Dockerfile for this image looks like:

FROM python:3.11.5-slim

WORKDIR /app

COPY ["model2.bin", "dv.bin", "./"]


We already built it and then pushed it to svizor/zoomcamp-model:3.11.5-slim.

`Note: You don't need to build this docker image, it's just for your reference.`

## Question 5

Download the base image svizor/zoomcamp-model:3.11.5-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

- 45 MB
- 130 MB
- 245 MB
- 330 MB

You can get this information when running `docker images` - it'll be in the "SIZE" column.

1. sudo docker run -it svizor/zoomcamp-model:3.11.5-slim
2. sudo docker images

Answer: 130 MB

### Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```
FROM svizor/zoomcamp-model:3.11.5-slim
**add your stuff here**
```

Now complete it:

- Install all the dependencies form the Pipenv file
- Copy your Flask script
- Run it with Gunicorn

After that, you can build your docker image.

## Question 6

Let's run your docker container!

After running it, score this client once again:

- url = "YOUR_URL"
- client = {"job": "management", "duration": 400, "poutcome": "success"}
- requests.post(url, json=client).json()

What's the probability that this client will get a subscription now?
- 0.287
- 0.530
- 0.757
- 0.960


```Dockerfile
FROM svizor/zoomcamp-model:3.11.5-slim

RUN pip install pipenv

WORKDIR /app

COPY ["Pipfile", "Pipfile.lock", "predict.py", "./"]

RUN pipenv install --system --deploy

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "predict:app"]
```

1. sudo docker build -t homework .

2. sudo docker run -it -p 9696:9696 homework

3. (other terminal) python3 request.py

Answer: 0.756